In [ ]:
## Day 1.
# ======================================================
# Walmart Data-Analytics Dashboard: PostgreSQL -> Python -> Google Sheets
# ======================================================

import psycopg2
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

# -----------------------------
# 1️⃣ Connect to PostgreSQL and run query
# -----------------------------
def run_query(query):
    conn = psycopg2.connect(
         dbname="Your_DB_Name",
         user="username",
         password="your_DB_password",
         host="localhost",
         port="5432"
    )
    cur = conn.cursor()
    cur.execute(query)
    data = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(data, columns=columns)
    cur.close()
    conn.close()
    return df

# -----------------------------
# 2️⃣ Google Sheets Authentication
# -----------------------------
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("sheets_Automation.json", scope)

## search on GPT how to create the json_keyfile, it will guide step-by-step.

client = gspread.authorize(creds)

# -----------------------------
# 3️⃣ Function to append query results to a single sheet
# -----------------------------
def append_query_to_sheet(query, worksheet_name, sheet_url):
    df = run_query(query)
    sheet = client.open_by_url(sheet_url).worksheet(worksheet_name)
    
    # Get the next empty row
    next_row = len(sheet.get_all_values()) + 2
    
    # Add a separator or header for new query (optional)
    sheet.update(f"A{next_row}", [[f"--- New Query Result ---"]])
    next_row += 1
    
    # Write DataFrame starting from the next empty row
    set_with_dataframe(sheet, df, row=next_row, include_index=False)
    
    print(f"✅ Query results appended successfully to {worksheet_name}!")

# -----------------------------
# 4️⃣ Example usage
# -----------------------------
sheet_url = "Google_sheet_url"
worksheet_name = "Worksheet_Name"

# First query
query1 = """
  Enter the SQL Query
"""
append_query_to_sheet(query1, worksheet_name, sheet_url)